https://github.com/wesm/pydata-book

In [4]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas import Series, DataFrame
import seaborn as sns

# used for example for random
from numpy import *
# for matplot
%matplotlib inline

In [5]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [6]:
from IPython.display import display
from IPython.display import Image

**=========================================================================================================================**

## Reshaping and Pivoting
here are a number of fundamental operations for rearranging tabular data. These are
alternatingly referred to as reshape or pivot operations.

## Reshaping with Hierarchical Indexing
Hierarchical indexing provides a consistent way to rearrange data in a DataFrame.
There are two primary actions:
```python
stack()
```
: this “rotates” or pivots from the columns in the data to the rows
```python
unstack()
```
: this pivots from the rows into the columns
<br><br>I’ll illustrate these operations through a series of examples. Consider a small DataFrame
with string arrays as row and column indexes:

In [7]:
data = DataFrame(np.arange(6).reshape((2, 3)),
                 index=pd.Index(['Ohio', 'Colorado'], name='state'),
                 columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [8]:
#Using the stack method on this data pivots the columns into the rows, producing a Series:
result = data.stack()
result
# From a hierarchically-indexed Series, you can rearrange the data back into a DataFrame with unstack:
result.unstack()

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


By default the innermost level is unstacked (same with stack). You can unstack a different
level by passing a level number or name:

In [9]:
#the most inner lvl unstack(normally it is by default the most inner lvl)
result.unstack(1)
#highest lvl unstack (you need to specify)
result.unstack(0)

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


Unstacking might introduce missing data if all of the values in the level aren’t found in
each of the subgroups:

In [10]:
s1 = Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = Series([4, 5, 6], index=['c', 'd', 'e'])

data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

data2.unstack()

#Stacking filters out missing data by default, so the operation is easily invertible:
data2.unstack().stack() #changing to float type
#as during unstacking we added NaN where values where missing if we want to unstack with the same values, 
#we need to remove Nan's during stucking back
data2.unstack().stack(dropna = True) 
data2.unstack().stack(dropna = False) 

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [11]:
result
df = DataFrame({'left': result, 'right': result + 5}, columns=pd.Index(['left', 'right'], name='side'))
df
#When unstacking in a DataFrame, the level unstacked becomes the lowest level in the result:
df.unstack('state')
df.unstack('state').stack('side')

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

**=========================================================================================================================**

## Pivoting “long” to “wide” Format

A common way to store multiple time series in databases and CSV is in so-called 
```python
long 
or 
stacked
```
format.
<br><br>
Data is frequently stored this way in relational databases like MySQL as a fixed schema
(column names and data types) allows the number of distinct values in the item column
to increase or decrease as data is added or deleted in the table. In the above example
date and item would usually be the primary keys (in relational database parlance),
offering both relational integrity and easier joins and programmatic queries in many
cases. The downside, of course, is that the data may not be easy to work with in long
format; you might prefer to have a DataFrame containing one column per distinct
item value indexed by timestamps in the date column. DataFrame’s pivot method performs
exactly this transformation:

In [12]:
data = pd.read_csv('CSV/macrodata.csv')
data.head()
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter,
                         name='date')
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
data = data.reindex(columns=columns)
#data.head(3)
#periods
#type(periods)
data.index = periods.to_timestamp('D', 'end')
#data.head(3)
ldata = data.stack().reset_index().rename(columns={0: 'value'})

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [13]:
ldata[:5]

,date,item,value
0,1959-03-31,realgdp,2710.349
1,1959-03-31,infl,0.000
2,1959-03-31,unemp,5.800
3,1959-06-30,realgdp,2778.801
4,1959-06-30,infl,2.340


In [26]:
help(DataFrame.pivot)

Help on function pivot in module pandas.core.frame:

pivot(self, index=None, columns=None, values=None)
    Reshape data (produce a "pivot" table) based on column values. Uses
    unique values from index / columns to form axes of the resulting
    DataFrame.
    
    Parameters
    ----------
    index : string or object, optional
        Column name to use to make new frame's index. If None, uses
        existing index.
    columns : string or object
        Column name to use to make new frame's columns
    values : string or object, optional
        Column name to use for populating new frame's values. If not
        specified, all remaining columns will be used and the result will
        have hierarchically indexed columns
    
    Returns
    -------
    pivoted : DataFrame
    
    See also
    --------
    DataFrame.pivot_table : generalization of pivot that can handle
        duplicate values for one index/column pair
    DataFrame.unstack : pivot based on the index values in

In [14]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted2 = ldata.pivot(index = 'date', columns = 'item', values ='value')
pivoted.head()
pivoted2.head()
pivoted2.equals(pivoted)

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2


item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2


True

The first two values passed are the columns to be used as the row and column index,
and finally an optional value column to fill the DataFrame. Suppose you had two value
columns that you wanted to reshape simultaneously:

In [15]:
ldata.head()

ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

,date,item,value
0,1959-03-31,realgdp,2710.349
1,1959-03-31,infl,0.000
2,1959-03-31,unemp,5.800
3,1959-06-30,realgdp,2778.801
4,1959-06-30,infl,2.340


,date,item,value,value2
0,1959-03-31,realgdp,2710.349,-1.218167
1,1959-03-31,infl,0.000,-0.314766
2,1959-03-31,unemp,5.800,0.005669
3,1959-06-30,realgdp,2778.801,-0.636645
4,1959-06-30,infl,2.340,-0.392279
5,1959-06-30,unemp,5.100,0.460180
6,1959-09-30,realgdp,2775.488,0.200679
7,1959-09-30,infl,2.740,0.576597
8,1959-09-30,unemp,5.300,-0.666258
9,1959-12-31,realgdp,2785.204,-0.246837


In [21]:
#By omitting the last argument, you obtain a DataFrame with hierarchical columns:
pivoted = ldata.pivot('date', 'item')
pivoted[:5]
pivoted['value'][:5]

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-03-31  0.00  2710.349   5.8 -0.314766 -1.218167  0.005669
1959-06-30  2.34  2778.801   5.1 -0.392279 -0.636645  0.460180
1959-09-30  2.74  2775.488   5.3  0.576597  0.200679 -0.666258
1959-12-31  0.27  2785.204   5.6  0.377944 -0.246837  0.278706
1960-03-31  2.31  2847.699   5.2  0.302358 -2.593779 -0.976240

item,infl,realgdp,unemp
date,,,
1959-03-31,0.00,2710.349,5.8
1959-06-30,2.34,2778.801,5.1
1959-09-30,2.74,2775.488,5.3
1959-12-31,0.27,2785.204,5.6
1960-03-31,2.31,2847.699,5.2


!! **Note that pivot is just a shortcut for creating a hierarchical index using set_index and
reshaping with unstack:**

In [30]:
#help(DataFrame.unstack)

In [29]:
ldata.head(4)
ldata.set_index(['date', 'item']).head(4)
unstacked = ldata.set_index(['date', 'item']).unstack('item')
unstacked[:4]

,date,item,value,value2
0,1959-03-31,realgdp,2710.349,-1.218167
1,1959-03-31,infl,0.000,-0.314766
2,1959-03-31,unemp,5.800,0.005669
3,1959-06-30,realgdp,2778.801,-0.636645


value    value2
date       item                       
1959-03-31 realgdp  2710.349 -1.218167
           infl        0.000 -0.314766
           unemp       5.800  0.005669
1959-06-30 realgdp  2778.801 -0.636645

value                    value2                    
item        infl   realgdp unemp      infl   realgdp     unemp
date                                                          
1959-03-31  0.00  2710.349   5.8 -0.314766 -1.218167  0.005669
1959-06-30  2.34  2778.801   5.1 -0.392279 -0.636645  0.460180
1959-09-30  2.74  2775.488   5.3  0.576597  0.200679 -0.666258
1959-12-31  0.27  2785.204   5.6  0.377944 -0.246837  0.278706

In [ ]:
reshaped.reset_index()

In [ ]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

In [31]:
help(pd.melt)


Help on function melt in module pandas.core.reshape.reshape:

melt(frame, id_vars=None, value_vars=None, var_name=None, value_name='value', col_level=None)
    "Unpivots" a DataFrame from wide format to long format, optionally
    leaving identifier variables set.
    
    This function is useful to massage a DataFrame into a format where one
    or more columns are identifier variables (`id_vars`), while all other
    columns, considered measured variables (`value_vars`), are "unpivoted" to
    the row axis, leaving just two non-identifier columns, 'variable' and
    'value'.
    
    
    Parameters
    ----------
    frame : DataFrame
    id_vars : tuple, list, or ndarray, optional
        Column(s) to use as identifier variables.
    value_vars : tuple, list, or ndarray, optional
        Column(s) to unpivot. If not specified, uses all columns that
        are not set as `id_vars`.
    var_name : scalar
        Name to use for the 'variable' column. If None it uses
        ``frame.

## MELT() function

pandas.melt(frame, id_vars=None, value_vars=None, var_name=None, value_name='value', col_level=None)[source]
<br><br>“Unpivots” a DataFrame from wide format to long format, optionally leaving identifier variables set.

This function is useful to massage a DataFrame into a format where one or more columns are identifier variables (id_vars), while all other columns, considered measured variables (value_vars), are “unpivoted” to the row axis, leaving just two non-identifier columns, ‘variable’ and ‘value’

In [32]:

df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

,A,B,C,key
0,1,4,7,foo
1,2,5,8,bar
2,3,6,9,baz


In [35]:
melted = pd.melt(df, id_vars =['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [36]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [39]:
#help(DataFrame.reset_index)

In [40]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [42]:
df.head(4)
pd.melt(df, id_vars = ['key'], value_vars = ['A', 'B'])

,A,B,C,key
0,1,4,7,foo
1,2,5,8,bar
2,3,6,9,baz


,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [43]:

pd.melt(df, value_vars=['A', 'B', 'C'])
pd.melt(df, value_vars=['key', 'A', 'B'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
